##<img src="https://databricks.com/wp-content/themes/databricks/assets/images/header_logo_2x.png" alt="logo" width="150"/> 

# Real World Evidence Data Analysis

<ol>
  <li> **Data**: We use a realistic simulation of patient EHR data using **[synthea](https://github.com/synthetichealth/synthea)**, for ~11,000 patients in Massachusetts </li>
  <li> **Ingestion and De-identification**: We use **pyspark** to read data from csv files, de-identify patient PII and write to Delta Lake</li>
  <li> **Database creation**: We then use delta tables to create a database of pateint recprds for subsequent data analysis</li>
</ol>
<div style="text-align: center; line-height: 0; padding-top: 9px;">
<img src="https://amir-hls.s3.us-east-2.amazonaws.com/public/rwe-uap.png" width=700>
</div>

## 1. Ingest data into Spark dataframes

In [3]:
from pyspark.sql import functions as F, Window
ehr_path = '/databricks-datasets/rwe/ehr/csv'
display(dbutils.fs.ls(ehr_path)) ## display list of files

path,name,size
dbfs:/databricks-datasets/rwe/ehr/csv/README.txt,README.txt,714
dbfs:/databricks-datasets/rwe/ehr/csv/allergies.csv,allergies.csv,639154
dbfs:/databricks-datasets/rwe/ehr/csv/careplans.csv,careplans.csv,6271344
dbfs:/databricks-datasets/rwe/ehr/csv/conditions.csv,conditions.csv,10703272
dbfs:/databricks-datasets/rwe/ehr/csv/encounters.csv,encounters.csv,86554610
dbfs:/databricks-datasets/rwe/ehr/csv/imaging_studies.csv,imaging_studies.csv,2041068
dbfs:/databricks-datasets/rwe/ehr/csv/immunizations.csv,immunizations.csv,19317094
dbfs:/databricks-datasets/rwe/ehr/csv/medications.csv,medications.csv,19424734
dbfs:/databricks-datasets/rwe/ehr/csv/observations.csv,observations.csv,310288424
dbfs:/databricks-datasets/rwe/ehr/csv/organizations.csv,organizations.csv,14769


<div style="text-align: center; line-height: 0; padding-top: 9px;">
<img src="https://amir-hls.s3.us-east-2.amazonaws.com/public/EHR/EHR1.png" width=1000>
</div>

In [5]:
# create a python dictionary of dataframes
ehr_dfs = {}
for path,name in [(f.path,f.name) for f in dbutils.fs.ls(ehr_path) if f.name !='README.txt']:
  df_name = name.replace('.csv','')
  ehr_dfs[df_name] = spark.read.csv(path,header=True,inferSchema=True)

# Display number of records in each table
out_str="<h2>There are {} tables in this collection with:</h2><br>".format(len(ehr_dfs))
for k in ehr_dfs:
  out_str+='{}: <i style="color:Tomato;">{}</i> records <br>'.format(k.upper(),ehr_dfs[k].count())

displayHTML(out_str)

There are 12 tables in this collection with: ALLERGIES: 5453 records CAREPLANS: 32743 records CONDITIONS: 84421 records ENCOUNTERS: 393234 records IMAGING_STUDIES: 8873 records IMMUNIZATIONS: 144874 records MEDICATIONS: 109142 records OBSERVATIONS: 2193029 records ORGANIZATIONS: 119 records PATIENTS: 11737 records PROCEDURES: 327171 records PROVIDERS: 119 records

## 2. De-identify Patient PII

<div style="text-align: center; line-height: 0; padding-top: 9px;">
<img src="https://amir-hls.s3.us-east-2.amazonaws.com/public/EHR/EHR2.png" width=1000>
</div>

In [8]:
from pyspark.sql.types import StringType, IntegerType, StructType, StructField
import hashlib

def encrypt_value(pii_col):
  sha_value = hashlib.sha1(pii_col.encode()).hexdigest()
  return sha_value

encrypt_value_udf = udf(encrypt_value, StringType())

In [9]:
pii_cols=['SSN','DRIVERS','PASSPORT','PREFIX','FIRST','LAST','SUFFIX','MAIDEN','BIRTHPLACE','ADDRESS']
patients_obfuscated = ehr_dfs['patients']

for c in pii_cols:
  patients_obfuscated = patients_obfuscated.withColumn(c,F.coalesce(c,F.lit('null'))).withColumn(c,encrypt_value_udf(c))
display(patients_obfuscated)

Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,MAIDEN,MARITAL,RACE,ETHNICITY,GENDER,BIRTHPLACE,ADDRESS,CITY,STATE,ZIP
a45a0f84-1d16-4517-9aa8-271c59f339fd,1979-04-05,null,58c9aa2823d8bd0137286d10f17b1a2899501213,2d879e0d1d74bef23bb908bd36d886376bc38556,597dc5f7a3af5aeb0248762bd780e04d6386632c,216585e29a7c365fb5ffc1a25f5532f89a24a60e,634bdd2e13fad70a98a14c4faffbfb808c5f9c7a,e1ad161f8d47634bc918333110337719fe27f4ff,2be88ca4242c76e8253ac62474851065032d6833,2be88ca4242c76e8253ac62474851065032d6833,M,white,american,M,65882d6205b5841a04bf945edd31901bc8a0c414,e59672f450931642ff2aa6a5104ea1d5c7de8ba1,Leominster,Massachusetts,1420
cc3e87e4-6172-45cb-81d2-9b5209617e00,1979-04-01,null,51e3759f7528852a42b1df63ab8cec3258c7d206,433aec80deb9ba0f0fc01a96ee7574ac8e95cab1,acdee29c785ef9279a5e4ce1e58a3a637cca1b59,216585e29a7c365fb5ffc1a25f5532f89a24a60e,46cec00fddd0320e8d50eb08a60b1674cabefc98,8864881fd4d06a53fc3e91466c380e33e38dc756,2be88ca4242c76e8253ac62474851065032d6833,2be88ca4242c76e8253ac62474851065032d6833,M,white,english,M,0b83eeba70c18a1eb0f0a8cf757b5bac43ccc30e,fd5b8e79dd5b413d0a559149d7f10570f5033b69,Newton,Massachusetts,2458
4edfaae9-eef0-4301-bf40-693ad26637bd,1948-01-09,null,837513b488d1b64ba0c38d1ce2ab07a7994f859a,5e63af06b6ac760834cd5873991829db305dc91d,7e159cbec75e8d1546112974ec07459d3bfc854e,37861bd65e4c2345a9cbcdcbfb928f49b68ed547,09bfb6c8657eeb75603e0234024190f7d4dba877,a84b7b081198cd23979b9d749db0fde3fb88d918,2be88ca4242c76e8253ac62474851065032d6833,5b94ad42e50d3b2304bcfc49edc68ee616ba57e6,M,white,french_canadian,F,cd83324112be6a599b54c3a3e9a5168a629347b8,6d1d78744023229022c02de4c51c2d1cc7cdef9b,Belmont,Massachusetts,2138
7ff0403d-6cc4-48a8-a0b1-ddb318b6017c,1956-01-12,null,46da089ab3528a7e5e16bca055e0bc6301b3edfa,d1097858f93c04332d8ce83ea95607b8cfe5f7cf,4b316ccedb0aaf87c74b903c832ad7de569fb1ee,216585e29a7c365fb5ffc1a25f5532f89a24a60e,bb4a004f8eb87d76c0f119e73ab77f1c25dd2a21,1f2041f08c36dcea407d1824541b7f177b3e0b70,2be88ca4242c76e8253ac62474851065032d6833,2be88ca4242c76e8253ac62474851065032d6833,M,white,german,M,f4c887690f3a692eff82aca66b29f493ebda69e4,736d1c3836585dc172fb5bea72571661d540da8b,Barnstable Town,Massachusetts,2601
2f905f97-cf1f-4f06-b54a-a047163c984b,1998-05-30,null,0038851e6f7b5018692127b2d04c094ff2ec822b,81df67d67f7e28cbbf79ef90687c46ec8829f089,281338c399d1ffff8d1d15feefa2e02ac09183a1,2b0e240bf5c614e439404dc4576d48e276530d50,e8e65fc267822da921085e9a816d853a58cafe68,fd6e41a5b0ece3b6b0e637174797f69f8d621c68,2be88ca4242c76e8253ac62474851065032d6833,2be88ca4242c76e8253ac62474851065032d6833,null,white,french_canadian,F,929bfea95183d2a58c3b874427bd0e32a1b7b65a,45411b2792f0a75939f97e75ec4a4d0824359f7e,Lynn,Massachusetts,1901
b9b551cf-adf6-4f28-a509-0e8003510001,1970-09-05,2014-03-15,802b6514c33a98a592126b66aabe4f84286cde39,e1a83794113cf3f3ab17b184443beb9dcf6efa4c,0c6f219c3a5276bb51d298ef4e7797b65100a919,216585e29a7c365fb5ffc1a25f5532f89a24a60e,c7b82af21a4f1157dfa55279aa82e96ac305377a,ca2561d1bd43a97454d89c0556e570dd439d81c9,2be88ca4242c76e8253ac62474851065032d6833,2be88ca4242c76e8253ac62474851065032d6833,M,black,dominican,M,13f0e559ac6d0788bdd64cfc5385af75d68191e6,90459ecec6a1d57be3cb12ead144d364b8cf5dd3,Springfield,Massachusetts,1013
e747f9b3-480b-4ce7-8bf7-87304f058933,1994-07-07,null,eacce36d7c9a0ab8b2cd2a7562e1c43a012f758e,0fe7b483d48b73a9da0b903fbcd3f26a5449bd85,7fb9b586dd75f3a0d39a89a9abf6bdca9c014e2c,2b0e240bf5c614e439404dc4576d48e276530d50,f1cd68fb90994f2cc9c2ad442ee48f0fe71ff163,a87c5b3eefe1b0b89d8738f6b46d81faa3cc47be,2be88ca4242c76e8253ac62474851065032d6833,2be88ca4242c76e8253ac62474851065032d6833,null,white,irish,F,ea39e9324ae9578641f5285bd836d1e0eb90618d,2cf264dc2cec442ac9d07e6ee8f5fd53c61e388e,Ware,Massachusetts,1082
61bb8fc6-b6ce-4725-92e4-8f2b637bf317,1990-09-28,null,79e386e9790013db62838021e1a155771c352edd,dc5d925614504ce21e37aadf764ec7a11cde3a50,19b62223648e3fed53a6fbd3c11f41e355464e2a,37861bd65e4c2345a9cbcdcbfb928f49b68ed547,5f126c5742546995c30f56bb6b1916d768624929,d68021ed5a

## 3. Write tables to Delta Lake

In [11]:
## Specify the path to delta tables on dbfs
delta_root_path = "dbfs:/tmp/rwe-ehr/delta"

## to ensure fresh start we delete the path if it already exist
dbutils.fs.rm(delta_root_path, recurse=True)

## Create enounters table with renamed columns
(
  ehr_dfs['encounters']
  .withColumnRenamed('Id','Enc_Id')
  .withColumnRenamed('START', 'START_TIME')
  .withColumnRenamed('END', 'END_TIME')
  .write.format('delta').save(delta_root_path + '/encounters')
)

## Create providers table with renamed columns
(
  ehr_dfs['providers']
  .withColumnRenamed('NAME','Provider_Name')
  .withColumnRenamed('Id','PROVIDER')
  .write.format('delta').save(delta_root_path + '/providers')
)

## Create organizations table with renamed columns
(
  ehr_dfs['organizations']
  .withColumnRenamed('NAME','Org_Name')
  .withColumnRenamed('Id','ORGANIZATION')
  .withColumnRenamed('ADDRESS', 'PROVIDER_ADDRESS')
  .withColumnRenamed('CITY', 'PROVIDER_CITY')
  .withColumnRenamed('STATE', 'PROVIDER_STATE')
  .withColumnRenamed('ZIP', 'PROVIDER_ZIP')
  .withColumnRenamed('GENDER', 'PROVIDER_GENDER')
  .write.format('delta').save(delta_root_path + '/organizations')
)

## Create patients from dataframe with obfuscated PII
(
  patients_obfuscated
  .write.format('delta').save(delta_root_path + '/patients')
)

<div style="text-align: center; line-height: 0; padding-top: 9px;">
<img src="https://amir-hls.s3.us-east-2.amazonaws.com/public/EHR/EHR3.png" width=1000>
</div>

In [13]:
patients = spark.read.format("delta").load(delta_root_path + '/patients').withColumnRenamed('Id', 'PATIENT')
encounters = spark.read.format("delta").load(delta_root_path + '/encounters').withColumnRenamed('PROVIDER', 'ORGANIZATION')
organizations = spark.read.format("delta").load(delta_root_path + '/organizations')

(
  encounters
  .join(patients, ['PATIENT'])
  .join(organizations, ['ORGANIZATION'])
  .write.format('delta').save(delta_root_path + '/patient_encounters')
)

## 4. Create database and tables

In [15]:
%sql
-- Create Database
CREATE DATABASE IF NOT EXISTS rwd
    COMMENT "Database for real world data"
    LOCATION "dbfs:/tmp/rwe-ehr/databases";

DROP TABLE IF EXISTS rwd.encounters;

-- Create encounters table

CREATE TABLE IF NOT EXISTS rwd.encounters
USING DELTA
LOCATION 'dbfs:/tmp/rwe-ehr/delta/encounters';

-- Create providers table

DROP TABLE IF EXISTS rwd.providers;

CREATE TABLE IF NOT EXISTS rwd.providers
USING DELTA
LOCATION 'dbfs:/tmp/rwe-ehr/delta/providers';

-- Create organizations table

DROP TABLE IF EXISTS rwd.organizations;

CREATE TABLE IF NOT EXISTS rwd.organizations
USING DELTA
LOCATION 'dbfs:/tmp/rwe-ehr/delta/organizations';

-- Create patients table

DROP TABLE IF EXISTS rwd.patients;

CREATE TABLE IF NOT EXISTS rwd.patients
USING DELTA
LOCATION 'dbfs:/tmp/rwe-ehr/delta/patients';

-- Create patient encounter table

DROP TABLE IF EXISTS rwd.patient_encounters;

CREATE TABLE IF NOT EXISTS rwd.patient_encounters
USING DELTA
LOCATION 'dbfs:/tmp/rwe-ehr/delta/patient_encounters';

In [16]:
%sql SELECT * FROM rwd.patient_encounters

ORGANIZATION,PATIENT,Enc_Id,START_TIME,STOP,ENCOUNTERCLASS,CODE,DESCRIPTION,COST,REASONCODE,REASONDESCRIPTION,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,MAIDEN,MARITAL,RACE,ETHNICITY,GENDER,BIRTHPLACE,ADDRESS,CITY,STATE,ZIP,Org_Name,PROVIDER_ADDRESS,PROVIDER_CITY,PROVIDER_STATE,PROVIDER_ZIP,PHONE,UTILIZATION
5103c940-0c08-392f-95cd-446e0cea042a,800f3a40-f8c9-4b05-a41a-8a125d4828ec,82299560-de32-4220-a626-d3c3d22438a7,2010-03-04T05:42:36.000+0000,2010-03-04T06:12:36.000+0000,wellness,185349003,Encounter for check up (procedure),129.16,null,null,1959-08-20,null,a151a1a90ec75a95364ddf1c0e0f14cb3ad34f5f,84049aa6b306988860d515071f45739e77633b7e,a565142ae927146903d2e65d7520c5c315958a0f,37861bd65e4c2345a9cbcdcbfb928f49b68ed547,511b4626b958b67d11e47f40c38c49260e12699c,66807bf0138309ea73825497a9b63695befcb644,2be88ca4242c76e8253ac62474851065032d6833,e3f3a94c4cf96c61c8162bf03d7e12cf569b4b09,M,asian,chinese,F,ec4798df10a58a6e1325a6561fafcfbf5568ac0e,e8b2b5a76271e240c4e95a030e9621b6b65d81d0,Springfield,Massachusetts,1013,SHRINERS' HOSPITAL FOR CHILDREN (THE),516 CAREW STREET,SPRINGFIELD,MA,1104,4137872000,51253
5103c940-0c08-392f-95cd-446e0cea042a,800f3a40-f8c9-4b05-a41a-8a125d4828ec,eafb15cf-4a59-4d81-82ca-1432cc995a46,2011-03-10T05:42:36.000+0000,2011-03-10T06:12:36.000+0000,wellness,185349003,Encounter for check up (procedure),129.16,null,null,1959-08-20,null,a151a1a90ec75a95364ddf1c0e0f14cb3ad34f5f,84049aa6b306988860d515071f45739e77633b7e,a565142ae927146903d2e65d7520c5c315958a0f,37861bd65e4c2345a9cbcdcbfb928f49b68ed547,511b4626b958b67d11e47f40c38c49260e12699c,66807bf0138309ea73825497a9b63695befcb644,2be88ca4242c76e8253ac62474851065032d6833,e3f3a94c4cf96c61c8162bf03d7e12cf569b4b09,M,asian,chinese,F,ec4798df10a58a6e1325a6561fafcfbf5568ac0e,e8b2b5a76271e240c4e95a030e9621b6b65d81d0,Springfield,Massachusetts,1013,SHRINERS' HOSPITAL FOR CHILDREN (THE),516 CAREW STREET,SPRINGFIELD,MA,1104,4137872000,51253
5103c940-0c08-392f-95cd-446e0cea042a,800f3a40-f8c9-4b05-a41a-8a125d4828ec,b8959784-cfbe-48f9-a784-2f14c30de085,2011-08-31T05:42:36.000+0000,2011-08-31T06:11:36.000+0000,ambulatory,185345009,Encounter for symptom,129.16,10509002,Acute bronchitis (disorder),1959-08-20,null,a151a1a90ec75a95364ddf1c0e0f14cb3ad34f5f,84049aa6b306988860d515071f45739e77633b7e,a565142ae927146903d2e65d7520c5c315958a0f,37861bd65e4c2345a9cbcdcbfb928f49b68ed547,511b4626b958b67d11e47f40c38c49260e12699c,66807bf0138309ea73825497a9b63695befcb644,2be88ca4242c76e8253ac62474851065032d6833,e3f3a94c4cf96c61c8162bf03d7e12cf569b4b09,M,asian,chinese,F,ec4798df10a58a6e1325a6561fafcfbf5568ac0e,e8b2b5a76271e240c4e95a030e9621b6b65d81d0,Springfield,Massachusetts,1013,SHRINERS' HOSPITAL FOR CHILDREN (THE),516 CAREW STREET,SPRINGFIELD,MA,1104,4137872000,51253
7ffe74ac-786d-3c6d-bcb9-323352f6149c,800f3a40-f8c9-4b05-a41a-8a125d4828ec,2e86d0c6-04ec-41af-b002-584ff98f3eff,2011-09-08T05:42:36.000+0000,2011-09-08T05:57:36.000+0000,urgentcare,371883000,Outpatient procedure (procedure),129.16,null,null,1959-08-20,null,a151a1a90ec75a95364ddf1c0e0f14cb3ad34f5f,84049aa6b306988860d515071f45739e77633b7e,a565142ae927146903d2e65d7520c5c315958a0f,37861bd65e4c2345a9cbcdcbfb928f49b68ed547,511b4626b958b67d11e47f40c38c49260e12699c,66807bf0138309ea73825497a9b63695befcb644,2be88ca4242c76e8253ac62474851065032d6833,e3f3a94c4cf96c61c8162bf03d7e12cf569b4b09,M,asian,chinese,F,ec4798df10a58a6e1325a6561fafcfbf5568ac0e,e8b2b5a76271e240c4e95a030e9621b6b65d81d0,Springfield,Massachusetts,1013,CONCENTRA URGENT CARE - SPRINGFIELD,140 CARANDO DRIVE,SPRINGFIELD,MA,1104,413-746-4006,1474
5103c940-0c08-392f-95cd-446e0cea042a,800f3a40-f8c9-4b05-a41a-8a125d4828ec,0f800e06-9504-4978-a5bb-3957e35c3b72,2012-03-15T05:42:36.000+0000,2012-03-15T06:12:36.000+0000,wellness,185349003,Encounter for check up (procedure),129.16,null,null,1959-08-20,null,a151a1a90ec75a95364ddf1c0e0f14cb3ad34f5f,84049aa6b306988860d515071f45739e77633b7e,a565142ae927146903d2e65d7520c5c315958a0f,37861bd65e4c2345a9cbcdcbfb928f49b68ed547,511

We can now use Delta's features for performance optimization. See this for more information see [ Delta Lake on Databricks ](https://docs.databricks.com/spark/latest/spark-sql/language-manual/optimize.html#optimize--delta-lake-on-databricks)

In [18]:
%sql OPTIMIZE rwd.patients ZORDER BY (BIRTHDATE, ZIP, GENDER, RACE)

path,metrics
null,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, List(minCubeSize(107374182400), List(0, 0), List(1, 2609768), 0, List(0, 0), 0), 0)"


In [19]:
%sql OPTIMIZE rwd.patient_encounters ZORDER BY (REASONDESCRIPTION, START_TIME, ZIP, PATIENT)

path,metrics
null,"List(1, 8, List(23997574, 23997574, 2.3997574E7, 1, 23997574), List(2154062, 3413974, 3244890.0, 8, 25959123), 0, List(minCubeSize(107374182400), List(0, 0), List(8, 25959123), 0, List(8, 25959123), 0), 1)"


In [20]:
%sql
use rwd

In [21]:
%sql
select * from patients

Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,MAIDEN,MARITAL,RACE,ETHNICITY,GENDER,BIRTHPLACE,ADDRESS,CITY,STATE,ZIP
a45a0f84-1d16-4517-9aa8-271c59f339fd,1979-04-05,null,58c9aa2823d8bd0137286d10f17b1a2899501213,2d879e0d1d74bef23bb908bd36d886376bc38556,597dc5f7a3af5aeb0248762bd780e04d6386632c,216585e29a7c365fb5ffc1a25f5532f89a24a60e,634bdd2e13fad70a98a14c4faffbfb808c5f9c7a,e1ad161f8d47634bc918333110337719fe27f4ff,2be88ca4242c76e8253ac62474851065032d6833,2be88ca4242c76e8253ac62474851065032d6833,M,white,american,M,65882d6205b5841a04bf945edd31901bc8a0c414,e59672f450931642ff2aa6a5104ea1d5c7de8ba1,Leominster,Massachusetts,1420
cc3e87e4-6172-45cb-81d2-9b5209617e00,1979-04-01,null,51e3759f7528852a42b1df63ab8cec3258c7d206,433aec80deb9ba0f0fc01a96ee7574ac8e95cab1,acdee29c785ef9279a5e4ce1e58a3a637cca1b59,216585e29a7c365fb5ffc1a25f5532f89a24a60e,46cec00fddd0320e8d50eb08a60b1674cabefc98,8864881fd4d06a53fc3e91466c380e33e38dc756,2be88ca4242c76e8253ac62474851065032d6833,2be88ca4242c76e8253ac62474851065032d6833,M,white,english,M,0b83eeba70c18a1eb0f0a8cf757b5bac43ccc30e,fd5b8e79dd5b413d0a559149d7f10570f5033b69,Newton,Massachusetts,2458
4edfaae9-eef0-4301-bf40-693ad26637bd,1948-01-09,null,837513b488d1b64ba0c38d1ce2ab07a7994f859a,5e63af06b6ac760834cd5873991829db305dc91d,7e159cbec75e8d1546112974ec07459d3bfc854e,37861bd65e4c2345a9cbcdcbfb928f49b68ed547,09bfb6c8657eeb75603e0234024190f7d4dba877,a84b7b081198cd23979b9d749db0fde3fb88d918,2be88ca4242c76e8253ac62474851065032d6833,5b94ad42e50d3b2304bcfc49edc68ee616ba57e6,M,white,french_canadian,F,cd83324112be6a599b54c3a3e9a5168a629347b8,6d1d78744023229022c02de4c51c2d1cc7cdef9b,Belmont,Massachusetts,2138
7ff0403d-6cc4-48a8-a0b1-ddb318b6017c,1956-01-12,null,46da089ab3528a7e5e16bca055e0bc6301b3edfa,d1097858f93c04332d8ce83ea95607b8cfe5f7cf,4b316ccedb0aaf87c74b903c832ad7de569fb1ee,216585e29a7c365fb5ffc1a25f5532f89a24a60e,bb4a004f8eb87d76c0f119e73ab77f1c25dd2a21,1f2041f08c36dcea407d1824541b7f177b3e0b70,2be88ca4242c76e8253ac62474851065032d6833,2be88ca4242c76e8253ac62474851065032d6833,M,white,german,M,f4c887690f3a692eff82aca66b29f493ebda69e4,736d1c3836585dc172fb5bea72571661d540da8b,Barnstable Town,Massachusetts,2601
2f905f97-cf1f-4f06-b54a-a047163c984b,1998-05-30,null,0038851e6f7b5018692127b2d04c094ff2ec822b,81df67d67f7e28cbbf79ef90687c46ec8829f089,281338c399d1ffff8d1d15feefa2e02ac09183a1,2b0e240bf5c614e439404dc4576d48e276530d50,e8e65fc267822da921085e9a816d853a58cafe68,fd6e41a5b0ece3b6b0e637174797f69f8d621c68,2be88ca4242c76e8253ac62474851065032d6833,2be88ca4242c76e8253ac62474851065032d6833,null,white,french_canadian,F,929bfea95183d2a58c3b874427bd0e32a1b7b65a,45411b2792f0a75939f97e75ec4a4d0824359f7e,Lynn,Massachusetts,1901
b9b551cf-adf6-4f28-a509-0e8003510001,1970-09-05,2014-03-15,802b6514c33a98a592126b66aabe4f84286cde39,e1a83794113cf3f3ab17b184443beb9dcf6efa4c,0c6f219c3a5276bb51d298ef4e7797b65100a919,216585e29a7c365fb5ffc1a25f5532f89a24a60e,c7b82af21a4f1157dfa55279aa82e96ac305377a,ca2561d1bd43a97454d89c0556e570dd439d81c9,2be88ca4242c76e8253ac62474851065032d6833,2be88ca4242c76e8253ac62474851065032d6833,M,black,dominican,M,13f0e559ac6d0788bdd64cfc5385af75d68191e6,90459ecec6a1d57be3cb12ead144d364b8cf5dd3,Springfield,Massachusetts,1013
e747f9b3-480b-4ce7-8bf7-87304f058933,1994-07-07,null,eacce36d7c9a0ab8b2cd2a7562e1c43a012f758e,0fe7b483d48b73a9da0b903fbcd3f26a5449bd85,7fb9b586dd75f3a0d39a89a9abf6bdca9c014e2c,2b0e240bf5c614e439404dc4576d48e276530d50,f1cd68fb90994f2cc9c2ad442ee48f0fe71ff163,a87c5b3eefe1b0b89d8738f6b46d81faa3cc47be,2be88ca4242c76e8253ac62474851065032d6833,2be88ca4242c76e8253ac62474851065032d6833,null,white,irish,F,ea39e9324ae9578641f5285bd836d1e0eb90618d,2cf264dc2cec442ac9d07e6ee8f5fd53c61e388e,Ware,Massachusetts,1082
61bb8fc6-b6ce-4725-92e4-8f2b637bf317,1990-09-28,null,79e386e9790013db62838021e1a155771c352edd,dc5d925614504ce21e37aadf764ec7a11cde3a50,19b62223648e3fed53a6fbd3c11f41e355464e2a,37861bd65e4c2345a9cbcdcbfb928f49b68ed547,5f126c5742546995c30f56bb6b1916d768624929,d68021ed5a

In [22]:
%sql

DESC HISTORY patients

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics
0,2020-08-27T18:30:08.000+0000,8149630156992488,swe.zhang@mail.mcgill.ca,WRITE,"Map(mode -> ErrorIfExists, partitionBy -> [])",null,List(1260302248242177),0827-181427-milch597,null,WriteSerializable,true,"Map(numFiles -> 1, numOutputBytes -> 2609768, numOutputRows -> 11737)"


In [23]:
%sql

DELETE
FROM patients
WHERE Id = 'a45a0f84-1d16-4517-9aa8-271c59f339fd'

In [24]:
%sql

DESC HISTORY patients

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics
1,2020-08-27T18:36:54.000+0000,8149630156992488,swe.zhang@mail.mcgill.ca,DELETE,"Map(predicate -> [""(`Id` = 'a45a0f84-1d16-4517-9aa8-271c59f339fd')""])",null,List(1260302248242177),0827-181427-milch597,0,WriteSerializable,false,"Map(numRemovedFiles -> 1, numDeletedRows -> 1, numAddedFiles -> 1, numCopiedRows -> 11736)"
0,2020-08-27T18:30:08.000+0000,8149630156992488,swe.zhang@mail.mcgill.ca,WRITE,"Map(mode -> ErrorIfExists, partitionBy -> [])",null,List(1260302248242177),0827-181427-milch597,null,WriteSerializable,true,"Map(numFiles -> 1, numOutputBytes -> 2609768, numOutputRows -> 11737)"


In [25]:
%sql
SELECT *
FROM patients@v1
WHERE Id = 'a45a0f84-1d16-4517-9aa8-271c59f339fd'

Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,MAIDEN,MARITAL,RACE,ETHNICITY,GENDER,BIRTHPLACE,ADDRESS,CITY,STATE,ZIP


In [26]:
%sql
SELECT *
FROM patients@v0
WHERE Id = 'a45a0f84-1d16-4517-9aa8-271c59f339fd'

Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,MAIDEN,MARITAL,RACE,ETHNICITY,GENDER,BIRTHPLACE,ADDRESS,CITY,STATE,ZIP
a45a0f84-1d16-4517-9aa8-271c59f339fd,1979-04-05,null,58c9aa2823d8bd0137286d10f17b1a2899501213,2d879e0d1d74bef23bb908bd36d886376bc38556,597dc5f7a3af5aeb0248762bd780e04d6386632c,216585e29a7c365fb5ffc1a25f5532f89a24a60e,634bdd2e13fad70a98a14c4faffbfb808c5f9c7a,e1ad161f8d47634bc918333110337719fe27f4ff,2be88ca4242c76e8253ac62474851065032d6833,2be88ca4242c76e8253ac62474851065032d6833,M,white,american,M,65882d6205b5841a04bf945edd31901bc8a0c414,e59672f450931642ff2aa6a5104ea1d5c7de8ba1,Leominster,Massachusetts,1420


In [27]:
%sql

--subquery 
with enc_org as(
  Select * from organizations org
  join encounters enc
  on enc.PROVIDER == org.ORGANIZATION
) 


select Org_Name,p.Provider_Name, count(*) as count
from enc_org as enc
join providers as p
on enc.ORGANIZATION == p.ORGANIZATION
group by 1,2
order by 3 desc
limit 10

Org_Name,Provider_Name,count
HALLMARK HEALTH SYSTEM,Lane Jacobi,17610
NORTH SHORE MEDICAL CENTER -,Sharyl Hilpert,14815
LOWELL GENERAL HOSPITAL,Ignacio Towne,13492
MILFORD REGIONAL MEDICAL CENTER,Wynell Olson,11507
SIGNATURE HEALTHCARE BROCKTON HOSPITAL,Barry Nicolas,10939
NEWTON-WELLESLEY HOSPITAL,Rebecca Labadie,10855
LAWRENCE GENERAL HOSPITAL,Tyron Torphy,10096
STURDY MEMORIAL HOSPITAL,Dagny Schoen,9521
WINCHESTER HOSPITAL,Andre Schneider,8739
NORWOOD HOSPITAL,Monica Kshlerin,8414


In [28]:
%sql
select EncounterClass, count(*) as cnt from encounters
group by EncounterClass
order by 2 desc

EncounterClass,cnt
ambulatory,162697
wellness,125441
outpatient,44626
urgentcare,26520
emergency,19771
inpatient,14179


We can set this ETL notebook [as a job](https://docs.databricks.com/jobs.html#create-a-job), that runs according to a given schedule.
Now proceed we create dashboard for quick data visualization. In the next notebook (`./01-rwe-dashboard.R`) we create a simple dashboard in `R`